In [1]:
import matplotlib.pyplot as plt
import numpy as np
from qiskit import QuantumCircuit, Aer, transpile
from qiskit.visualization import plot_histogram
from math import gcd
from numpy.random import randint
import pandas as pd
from fractions import Fraction
print("Imports Successful")

Imports Successful


In [2]:
def c_amod15(a, power):
    """Controlled multiplication by a mod 15"""
    if a not in [2,4,7,8,11,13]:
        raise ValueError("'a' must be 2,4,7,8,11 or 13")
    U = QuantumCircuit(4)
    for _iteration in range(power):
        if a in [2,13]:
            U.swap(2,3)
            U.swap(1,2)
            U.swap(0,1)
        if a in [7,8]:
            U.swap(0,1)
            U.swap(1,2)
            U.swap(2,3)
        if a in [4, 11]:
            U.swap(1,3)
            U.swap(0,2)
        if a in [7,11,13]:
            for q in range(4):
                U.x(q)
    U = U.to_gate()
    U.name = f"{a}^{power} mod 15"
    c_U = U.control()
    return c_U

In [3]:
# Specify variables
N_COUNT = 4  # number of counting qubits
a = 7

In [4]:
def qft_dagger(n):
    """n-qubit QFTdagger the first n qubits in circ"""
    qc = QuantumCircuit(n)
    # Don't forget the Swaps!
    for qubit in range(n//2):
        qc.swap(qubit, n-qubit-1)
    for j in range(n):
        for m in range(j):
            qc.cp(-np.pi/float(2**(j-m)), m, j)
        qc.h(j)
    qc.name = "QFT†"
    return qc

In [5]:
# Create QuantumCircuit with N_COUNT counting qubits
# plus 4 qubits for U to act on
qc = QuantumCircuit(N_COUNT + 4, N_COUNT)

# Initialize counting qubits
# in state |+>
for q in range(N_COUNT):
    qc.h(q)

# And auxiliary register in state |1>
qc.x(N_COUNT)

# Do controlled-U operations
for q in range(N_COUNT):
    qc.append(c_amod15(a, 2**q),
             [q] + [i+N_COUNT for i in range(4)])

# Do inverse-QFT
qc.append(qft_dagger(N_COUNT), range(N_COUNT))

# Measure circuit
qc.measure(range(N_COUNT), range(N_COUNT))
qc.draw(fold=-1)  # -1 means 'do not fold'

┌───┐                                                            ┌───────┐┌─┐         
q_0: ┤ H ├───────■────────────────────────────────────────────────────┤0      ├┤M├─────────
     ├───┤       │                                                    │       │└╥┘┌─┐      
q_1: ┤ H ├───────┼──────────────■─────────────────────────────────────┤1      ├─╫─┤M├──────
     ├───┤       │              │                                     │  QFT† │ ║ └╥┘┌─┐   
q_2: ┤ H ├───────┼──────────────┼──────────────■──────────────────────┤2      ├─╫──╫─┤M├───
     ├───┤       │              │              │                      │       │ ║  ║ └╥┘┌─┐
q_3: ┤ H ├───────┼──────────────┼──────────────┼──────────────■───────┤3      ├─╫──╫──╫─┤M├
     ├───┤┌──────┴──────┐┌──────┴──────┐┌──────┴──────┐┌──────┴──────┐└───────┘ ║  ║  ║ └╥┘
q_4: ┤ X ├┤0            ├┤0            ├┤0            ├┤0            ├──────────╫──╫──╫──╫─
     └───┘│             ││             ││             ││             │          ║  ║  ║  ║ 
q_5: ─────┤1            ├┤1            ├┤1            ├┤1            ├──────────╫──╫──╫──╫─
          │  7^1 mod 15 ││  7^2 mod 15 ││  7^4 mod 15 ││  7^8 mod 15 │          ║  ║  ║  ║ 
q_6: ─────┤2            ├┤2            ├┤2            ├┤2            ├──────────╫──╫──╫──╫─
          │             ││             ││             ││             │          ║  ║  ║  ║ 
q_7: ─────┤3            ├┤3            ├┤3            ├┤3            ├──────────╫──╫──╫──╫─
          └─────────────┘└─────────────┘└─────────────┘└─────────────┘          ║  ║  ║  ║ 
c: 4/═══════════════════════════════════════════════════════════════════════════╩══╩══╩══╩═
                                                                                0  1  2  3

In [6]:
trans_qc = transpile(
    qc,
    basis_gates=['x', 'h', 'rz', 'add', 'cx'],
)

In [7]:
# Get OpenQASM files
qubits = trans_qc.num_qubits
# Prevent printing of qasm: formatted=False
trans_qc.qasm(formatted=False, filename="./raw_qasm/shor_7_mod_15_n{}_from_python.qasm".format(qubits))
print("Done for counting qubits = {}".format(N_COUNT))

Done for counting qubits = 4
